# Mixed Data DDM

DDM using both patient and matched control data

In [2]:
%matplotlib inline
%cd ..

import warnings; warnings.filterwarnings('ignore')

/Users/celefthe/Programming/projects/language_decision


#### Start by parsing the .mat files from the matched controls

In [2]:
from utils import matparser, data_compiler
import glob

data_dir = 'data/controls/'

matparser.parse_dir(data_dir)

out_dir = "data/controls.csv"
data_compiler.compile_dir(data_dir, out_dir)

'data/controls.csv'

Clean up - remove *nan* entries as these cause hddm to fail

In [3]:
!cat data/controls.csv | grep -v nan > data/controls_clean.csv 

Merge patient and matched control data to single .csv file

First, create tagged versions of the csv files - include "tag" column to differentiate between patient and control

In [36]:
!cat data/patients_clean.csv | sed 1d | awk -v d="patient" -F"," 'BEGIN { OFS = "," } {$5=d; print}' | tr -d $'\r' > data/patients_tagged.csv
!cat data/controls_clean.csv | sed 1d | awk -v d="control" -F"," 'BEGIN { OFS = "," } {$5=d; print}' | tr -d $'\r' > data/controls_tagged.csv

!echo "response,rt,subj_idx,stim,subj_type" > data/combined_clean.csv
!cat data/patients_tagged.csv >> data/combined_clean.csv; sed 1d data/controls_tagged.csv >> data/combined_clean.csv

## Build HDDM model

In [3]:
import hddm

data = hddm.load_csv('data/combined_clean.csv')

model = hddm.HDDM(data, depends_on={'v': ['stim', 'subj_type'], 'a': 'subj_type'})
model.find_starting_values()
model.sample(6000, burn=20)

After 7.000000 retries, still no good fit found.


ZeroProbability: Stochastic wfpt(18439.control).1.5451359127328033's value is outside its support,
 or it forbids its parents' current values.